ДЗ № 8. Рекуррентные нейронные сети RNN LSTM GRU
---

**Материалы**

У вас лекционный ноутбук в архиве он с картинками и презентация тоже есть в архиве

**Задание**

На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для задачи сантимент анализа на данных с вебинара
1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN и (RNN -> CNN)
4. сдлать выводы что получилось лучше

In [2]:
# загрузка файла с локального диска
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv
Saving val.csv to val.csv


In [2]:
#!pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32916 sha256=10cf0ee67aa80fef4ce030bb800907d3cb90d305c274382cb3ec30694c81092b
  Stored in directory: c:\users\alex\appdata\local\pip\cache\wheels\eb\03\0d\3bd31c983789aeb0b4d5e2ca48590288d9db1586cf5f225062
Successfully built stop-words


In [4]:
#!pip install pymorphy2

  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=4b513e35876b5820d034f502fdb2ac4ed5562228edb8163dc503027d74241ee4
  Stored in directory: c:\users\alex\appdata\local\pip\cache\wheels\56\ea\58\ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [5]:
import pandas as pd
pd.set_option('max_colwidth', 120)
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping

ModuleNotFoundError: No module named 'keras'

# Обработка данных

In [90]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_val = pd.read_csv("val.csv")

In [91]:
df_train.head(5)

,id,text,class
0,0,"@alisachachka не уезжаааааааай. :(❤ я тоже не хочу, чтобы ты уезжала.",0
1,1,"RT @GalyginVadim: Ребята и девчата!\nВсе в кино!!! ""Вот Это Любовь!""\nСегодня! Завтра! И потом!)))))\n#вотэтолюбовь",1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретвит((((( RT,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Запретный плод. :),1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса не боится мороза и .......)),1


In [92]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [93]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [94]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [95]:
y_train = df_train['class'].values
y_val = df_val['class'].values

# Свёрточная сеть

In [96]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    output_dim=128, 
                    input_length=training_length, 
                    trainable=True,
                    mask_zero=True)
         )
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )

In [97]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 98s 304ms/step - loss: 0.6094 - accuracy: 0.6501 - val_loss: 0.4833 - val_accuracy: 0.7565


In [98]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 5ms/step - loss: 0.5073 - accuracy: 0.7468


Test score: 0.5072908997535706
Test accuracy: 0.7467706799507141


In [99]:
compare_result = pd.DataFrame(columns=['Model', 'Score', 'Accuracy'])
compare_result.loc[0] = ['CNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.507291,0.746771


# RNN

## Простая рекуррентная сеть

In [100]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True)
         )
model.add(Masking(mask_value=0.0))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )

In [101]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 39s 116ms/step - loss: 0.6172 - accuracy: 0.6378 - val_loss: 0.4939 - val_accuracy: 0.7560


In [102]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 6ms/step - loss: 0.5070 - accuracy: 0.7443


Test score: 0.5070378184318542
Test accuracy: 0.7443019151687622


In [103]:
compare_result.loc[1] = ['RNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.507291,0.746771
1,RNN,0.507038,0.744302


## LSTM

In [104]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True)
          )
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy']
              )

In [105]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train,
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 65s 194ms/step - loss: 0.6115 - accuracy: 0.6477 - val_loss: 0.4960 - val_accuracy: 0.7540


In [106]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 13ms/step - loss: 0.5137 - accuracy: 0.7395


Test score: 0.5136818885803223
Test accuracy: 0.7395406365394592


In [107]:
compare_result.loc[2] = ['LSTM', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.507291,0.746771
1,RNN,0.507038,0.744302
2,LSTM,0.513682,0.739541


## GRU

In [108]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
                    input_length=training_length,
                    output_dim=30,
                    trainable=True,
                    mask_zero=True)
         )
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']
              )

In [109]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, y_train,batch_size=512,epochs=10,verbose=1,validation_split=0.1,callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 61s 180ms/step - loss: 0.6049 - accuracy: 0.6557 - val_loss: 0.5001 - val_accuracy: 0.7513


In [110]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 12ms/step - loss: 0.5169 - accuracy: 0.7377


Test score: 0.5168968439102173
Test accuracy: 0.7377331256866455


In [111]:
compare_result.loc[3] = ['GRU', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.507291,0.746771
1,RNN,0.507038,0.744302
2,LSTM,0.513682,0.739541
3,GRU,0.516897,0.737733


In [112]:
compare_result

,Model,Score,Accuracy
0,CNN,0.507291,0.746771
1,RNN,0.507038,0.744302
2,LSTM,0.513682,0.739541
3,GRU,0.516897,0.737733


# Совместные архитектуры CNN -> RNN

## CNN-SimpleRNN

In [113]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True)
          )
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train, 
                    batch_size=512, 
                    epochs=10, verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 35s 104ms/step - loss: 0.6170 - accuracy: 0.6316 - val_loss: 0.4897 - val_accuracy: 0.7605


In [114]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 5ms/step - loss: 0.5064 - accuracy: 0.7443


Test score: 0.5063502788543701
Test accuracy: 0.7443459630012512


In [115]:
compare_result.loc[4] = ['CNN-SimpleRNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.507291,0.746771
1,RNN,0.507038,0.744302
2,LSTM,0.513682,0.739541
3,GRU,0.516897,0.737733
4,CNN-SimpleRNN,0.506350,0.744346


## CNN - LSTM

In [116]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True)
          )
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train,
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 57s 169ms/step - loss: 0.6017 - accuracy: 0.6568 - val_loss: 0.4854 - val_accuracy: 0.7592


In [117]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 11ms/step - loss: 0.5086 - accuracy: 0.7437


Test score: 0.5086171627044678
Test accuracy: 0.743684709072113


In [118]:
compare_result.loc[5] = ['CNN - LSTM', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.507291,0.746771
1,RNN,0.507038,0.744302
2,LSTM,0.513682,0.739541
3,GRU,0.516897,0.737733
4,CNN-SimpleRNN,0.506350,0.744346
5,CNN - LSTM,0.508617,0.743685


## CNN-GRU

In [119]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
                    input_length=training_length,
                    output_dim=30,
                    trainable=True, 
                    mask_zero=True)
          )
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 55s 161ms/step - loss: 0.6031 - accuracy: 0.6536 - val_loss: 0.4900 - val_accuracy: 0.7555


In [120]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 10ms/step - loss: 0.5281 - accuracy: 0.7397


Test score: 0.5281475782394409
Test accuracy: 0.7396728992462158


In [121]:
compare_result.loc[6] = ['CNN-GRU', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.507291,0.746771
1,RNN,0.507038,0.744302
2,LSTM,0.513682,0.739541
3,GRU,0.516897,0.737733
4,CNN-SimpleRNN,0.506350,0.744346
5,CNN - LSTM,0.508617,0.743685
6,CNN-GRU,0.528148,0.739673


# Совместные архитектуры RNN -> CNN

## SimpleRNN-CNN


In [122]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(SimpleRNN(64, return_sequences=True))
model.add(Conv1D(64, 3))
model.add(Activation("relu"))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 40s 118ms/step - loss: 0.6820 - accuracy: 0.5291 - val_loss: 0.6590 - val_accuracy: 0.5609


In [123]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 8ms/step - loss: 0.6644 - accuracy: 0.5517


Test score: 0.6644251942634583
Test accuracy: 0.5516571998596191


In [124]:
compare_result.loc[7] = ['SimpleRNN-CNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.507291,0.746771
1,RNN,0.507038,0.744302
2,LSTM,0.513682,0.739541
3,GRU,0.516897,0.737733
4,CNN-SimpleRNN,0.506350,0.744346
5,CNN - LSTM,0.508617,0.743685
6,CNN-GRU,0.528148,0.739673
7,SimpleRNN-CNN,0.664425,0.551657


## LSTM - CNN

In [125]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30)
          )
model.add(LSTM(64, recurrent_dropout=0.2, return_sequences=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train,
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 59s 172ms/step - loss: 0.6834 - accuracy: 0.5261 - val_loss: 0.6610 - val_accuracy: 0.5559


In [126]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 13ms/step - loss: 0.6652 - accuracy: 0.5472


Test score: 0.6652059555053711
Test accuracy: 0.5472080707550049


In [127]:
compare_result.loc[8] = ['LSTM - CNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.507291,0.746771
1,RNN,0.507038,0.744302
2,LSTM,0.513682,0.739541
3,GRU,0.516897,0.737733
4,CNN-SimpleRNN,0.506350,0.744346
5,CNN - LSTM,0.508617,0.743685
6,CNN-GRU,0.528148,0.739673
7,SimpleRNN-CNN,0.664425,0.551657
8,LSTM - CNN,0.665206,0.547208


## GRU-CNN

In [128]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
                    input_length=training_length,
                    output_dim=30,
                    trainable=True, 
                    mask_zero=True)
          )
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2, return_sequences=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 61s 179ms/step - loss: 0.6818 - accuracy: 0.5265 - val_loss: 0.6614 - val_accuracy: 0.5556


In [129]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 13ms/step - loss: 0.6648 - accuracy: 0.5472


Test score: 0.6648109555244446
Test accuracy: 0.5472010970115662


In [131]:
compare_result.loc[9] = ['GRU - CNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.507291,0.746771
1,RNN,0.507038,0.744302
2,LSTM,0.513682,0.739541
3,GRU,0.516897,0.737733
4,CNN-SimpleRNN,0.506350,0.744346
5,CNN - LSTM,0.508617,0.743685
6,CNN-GRU,0.528148,0.739673
7,SimpleRNN-CNN,0.664425,0.551657
8,LSTM - CNN,0.665206,0.547208
9,GRU - CNN,0.664811,0.547201


**Вывод**: все модели кроме RNN -> CNN показывают примерно одинаковую результативность.